In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient
from google_trans_new import google_translator
from langdetect import detect, DetectorFactory
from datetime import datetime
import csv
DetectorFactory.seed = 0 


In [2]:
client = MongoClient('mongodb://192.168.1.11:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [4]:
url = 'http://192.168.1.11:3000'

In [5]:
driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [6]:
driver.get(url)
driver.maximize_window()

In [7]:
driver.find_element(By.XPATH, '/html/body/nav/a[2]').click()
time.sleep(30)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/nav/a[2]"}
  (Session info: chrome=123.0.6312.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00FA4CC3+225091]
	(No symbol) [0x00ED4E11]
	(No symbol) [0x00D79A7A]
	(No symbol) [0x00DB175B]
	(No symbol) [0x00DB188B]
	(No symbol) [0x00DE7882]
	(No symbol) [0x00DCF5A4]
	(No symbol) [0x00DE5CB0]
	(No symbol) [0x00DCF2F6]
	(No symbol) [0x00DA79B9]
	(No symbol) [0x00DA879D]
	sqlite3_dbdata_init [0x01419A63+4064547]
	sqlite3_dbdata_init [0x0142106A+4094762]
	sqlite3_dbdata_init [0x0141B968+4072488]
	sqlite3_dbdata_init [0x0111C9C9+930953]
	(No symbol) [0x00EE07E4]
	(No symbol) [0x00EDAD08]
	(No symbol) [0x00EDAE31]
	(No symbol) [0x00ECCAA0]
	BaseThreadInitThunk [0x755E6839+25]
	RtlGetFullPathName_UEx [0x777E906F+1215]
	RtlGetFullPathName_UEx [

In [ ]:
html = driver.page_source
soup = bs(html, 'html.parser')
div_novels = soup.find('div', class_='novelOptions')
list_novels = div_novels.find_all('a')
data_novelas = {"nombre":[], "url":[]}
for chap in list_novels:
    data_novelas['nombre'].append(chap.getText())
    data_novelas['url'].append(chap.get('href'))
df_novelas = pd.DataFrame(data_novelas)


In [ ]:
df_novelas

In [ ]:
def traducir(texto):
    contenido_p = ''
    while True:
        try:
            contenido_p = ts.translate_text(
                texto, translator='bing', to_language='es')
            break
        except Exception as e:
            # print(e)
            try:
                contenido_p = ts.translate_text(
                    texto, translator='google', to_language='es')
                break
            except Exception as e:
                # print(e)
                try:
                    translator = google_translator() 
                    contenido_p = translator.translate(texto, lang_tgt='es')
                except Exception as e:
                    # print(e)
                    pass
                pass
            pass
    return contenido_p

In [ ]:
def enviar_contenido_capitulo(novel_id,capitulo_id,texto_capitulo):
    url_contenido_capitulos = "http://192.168.1.11:8000/api/contenidocapitulos/"
    headers = {
        'Content-Type': 'application/json'
    }
    payload_chapter = {
        "novela_id": novel_id,
        "capitulo_id": capitulo_id,
        "texto": texto_capitulo,
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat()
    }

    try:
        res_cap = requests.post(url_contenido_capitulos, headers=headers, json=payload_chapter)
        res_cap.raise_for_status()
        return res_cap.json()['_id']
    except requests.exceptions.RequestException as e:
        print(f"Error al enviar el contenido del capítulo: {e}")
        return None

In [ ]:
def manejar_driver(url, driver, novel_id):
    """
    Maneja el proceso de extracción de contenido de capítulos de una novela.

    Args:
        url (str): La URL de la novela.
        driver (webdriver): El controlador del navegador web.
        novel_id (str): El ID de la novela.

    Returns:
        None
    """
    driver.get(url)
    list_chapter = driver.find_elements(By.CLASS_NAME, 'capitulo')

    # Obtener una lista de capítulos existentes en la base de datos
    coleccion = db['app_contenidocapitulo']
    capitulos_existentes = coleccion.find({'novela_id': novel_id}, {'capitulo_id': 1, '_id': 0})
    capitulos_existentes = [doc['capitulo_id'] for doc in capitulos_existentes]

    for chapter in list_chapter:
        capitulo_id = chapter.get_attribute('id')
        if capitulo_id not in capitulos_existentes:
            print(f"Procesando capítulo: {capitulo_id}", end=" | ")
            chapter.click()
            time.sleep(3)  # Esperar a que se cargue el contenido del capítulo

            # Cambiar el contexto al iframe
            iframe = driver.find_element(By.XPATH, "/html/body/div/div/div[1]/iframe")
            driver.switch_to.frame(iframe)

            try:
                html_frame = driver.page_source
                soup_frame = bs(html_frame, 'html.parser')
                div_chap_content = soup_frame.find('div', id='chr-content')
                p_chap_content = div_chap_content.find_all('p')
                chap_content = [p.getText().strip().rstrip() for p in p_chap_content]

                texto_capitulo = ''
                for parrafo in chap_content:
                    if parrafo != '':
                        if len(parrafo) > 4000:
                            texto_traducido = '. '.join([traducir(x) for x in parrafo.split('. ') if x != ''])
                            texto_capitulo += f"<p>{texto_traducido}</p>"
                        else:
                            texto_traducido = traducir(parrafo)
                            texto_capitulo += f"<p>{texto_traducido}</p>"

                _id = enviar_contenido_capitulo(novel_id=novel_id, capitulo_id=capitulo_id, texto_capitulo=texto_capitulo)
                print(f"ID del capítulo insertado: {_id}")
            except Exception as e:
                print(f"El capitulo {capitulo_id} ha presentado problemas al momento de acceder.")
                pass

            # Volver al contexto principal de la página web
            driver.switch_to.default_content()
        # else:
        #     print(f"El capítulo {capitulo_id} ya existe en la base de datos. Omitiendo...", end=" | ")
        # break
    

In [ ]:
for idx, novela in df_novelas.iterrows():
    novel_id = novela.url.split('/')[-1]
    print(novel_id)
    time.sleep(4)
    manejar_driver(url=f"{url}{novela.url}", driver=driver, novel_id=novel_id)
    # break

In [ ]:
driver.quit()